### Basic imports and wget function -- needs to be done every session

In [1]:
from subprocess import call
import json
import gmaps
import os
import math

gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])
info_box_template = """
<header>Aircraft {Id}:</header>
<dl>
<dt>Airline</dt><dd>{Op}</dd>
<dt>Model</dt><dd>{Mdl}</dd>
<dt>From</dt><dd>{From}</dd>
<dt>Destination</dt><dd>{To}</dd>
</dl>
"""
marker_location = [0,0]

def cleanData(data):
    for aircraft in data['acList']:
        frm = aircraft.get('From', 'no from')
        dest = aircraft.get('To', 'no dest')
        airline = aircraft.get('Op', 'no airline')
        model = aircraft.get('Mdl', 'no model')
        if(frm == 'no from'):
            aircraft['From'] = 'Not Avaliable'
        if(dest == 'no dest'):
            aircraft['To'] = 'Not Avaliable'
        if(airline == 'no airline'):
            aircraft['Op'] = 'Not Avaliable'
        if(model == 'no model'):
            aircraft['Mdl'] = 'Not Avaliable'

def getNewAircraftData(URL, jsonFile):
    call(["wget", "-O", jsonFile, URL])
    json_data = open(jsonFile).read()
    aircraftData = json.loads(json_data)
    return aircraftData

def grabData():
    URL = "http://public-api.adsbexchange.com/VirtualRadar/AircraftList.json"
    jsonFile = "AircraftList.json"
    data = getNewAircraftData(URL, jsonFile)
    cleanData(data)
    return data

def distance(lat1, lon1, lat2, lon2):

    rad_e = (3959)
    rad_lat1 = math.radians(lat1)
    rad_lon1 = math.radians(lon1)
    rad_lat2 = math.radians(lat2)
    rad_lon2 = math.radians(lon2)

    change_in_lat = rad_lat2 - rad_lat1
    change_in_lon = rad_lon2 - rad_lon1

    a = ((math.sin(change_in_lat/2)**2)+(math.cos(rad_lat1)*(math.cos(rad_lat2))*(math.sin(change_in_lon/2)**2)))
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = rad_e * c
    return d

##data = grabData()

### Tracks aircraft in a "square" around a center point

In [2]:
lovelandSkiArea = (39.680395, -105.897961)
myHouse = (39.921991, -104.955664)
regis = (39.788484, -105.033411)
grab_data = True
if(grab_data):
    data = grabData()
center_point = regis
dist = 1 ##+/- lat/lon cordanants
count = 0
map = gmaps.figure(center=center_point, zoom_level=10, map_type='HYBRID')
for aircraft in data['acList']:
    lat = aircraft.get('Lat', 'no lat')
    lon = aircraft.get('Long', 'no lon')
    if(lat == 'no lat' or lon == 'no lon'):
        continue
    if((lat > (center_point[0] - dist)) and (lat < (center_point[0] + dist))):
        if((lon > (center_point[1] - dist)) and (lon < (center_point[1] + dist))):
            marker_location[0] = lat
            marker_location[1] = lon
            aircraft_info = [info_box_template.format(**aircraft)]
            marker = gmaps.marker_layer([marker_location], info_box_content=aircraft_info)
            map.add_layer(marker)
            count += 1
print(count)

map


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Targets a single airline

In [ ]:
grab_data = True
if(grab_data):
    data = grabData()
target_airline = 'United Airlines'
map = gmaps.figure(map_type='HYBRID')
for aircraft in data['acList']:
    airline = aircraft.get('Op', 'no airline')
    lat = aircraft.get('Lat', 'no lat')
    lon = aircraft.get('Long', 'no lon')
    if(lat == 'no lat' or lon == 'no lon'):
        continue
    if(airline.lower().__contains__(target_airline.lower())):
        marker_location[0] = lat
        marker_location[1] = lon
        aircraft_info = [info_box_template.format(**aircraft)]
        marker = gmaps.marker_layer([marker_location], info_box_content=aircraft_info)
        map.add_layer(marker)

map


### Library of airports

In [ ]:
DIA = 'KDEN Denver, United States'
MSP = 'KMSP Minneapolis-St Paul International/Wold-Chamberlain, Minneapolis, United States'
LAX = 'KLAX Los Angeles, United States'
SFO = 'KSFO San Francisco, United States'
LGA = 'KLGA La Guardia, New York, United States'
EWR = 'KEWR Newark Liberty, United States'
DUB = 'EIDW Dublin, Ireland'
MDW = 'KMDW Chicago Midway, United States'
ORD = 'KORD Chicago O\'Hare, United States'
SAT = 'KSAT San Antonio, United States'
SAN = 'KSAN San Diego, United States'
IAD = 'KIAD Washington Dulles, United States'
MCO = 'KMCO Orlando, United States'
DTW = 'KDTW Detroit Metropolitan Wayne County, United States'
GHP = 'EGPH Edinburgh, United Kingdom'

### Targets the destination airport only

In [ ]:
grab_data = True
if(grab_data):
    data = grabData()
target_dest = LAX
map = gmaps.figure(map_type='HYBRID')
for aircraft in data['acList']:
    dest = aircraft.get('To', 'no dest')
    lat = aircraft.get('Lat', 'no lat')
    lon = aircraft.get('Long', 'no lon')
    if(lat == 'no lat' or lon == 'no lon'):
        continue
    if(dest == target_dest):
        marker_location[0] = lat
        marker_location[1] = lon
        aircraft_info = [info_box_template.format(**aircraft)]
        marker = gmaps.marker_layer([marker_location], info_box_content=aircraft_info)
        map.add_layer(marker)

map


### Targets the airport left from only

In [ ]:
grab_data = True
if(grab_data):
    data = grabData()
target_from = EWR
map = gmaps.figure(map_type='HYBRID')
for aircraft in data['acList']:
    frm = aircraft.get('From', 'no frm')
    lat = aircraft.get('Lat', 'no lat')
    lon = aircraft.get('Long', 'no lon')
    if(lat == 'no lat' or lon == 'no lon'):
        continue
    if(frm == target_from):
        marker_location[0] = lat
        marker_location[1] = lon
        aircraft_info = [info_box_template.format(**aircraft)]
        marker = gmaps.marker_layer([marker_location], info_box_content=aircraft_info)
        map.add_layer(marker)

map


### Targets aircraft to AND from somewhere

In [ ]:
grab_data = True
if(grab_data):
    data = grabData()
target_from = DTW
target_dest = DIA
map = gmaps.figure(map_type='HYBRID')
for aircraft in data['acList']:
    dest = aircraft.get('To', 'no dest')
    frm = aircraft.get('From', 'no frm')
    lat = aircraft.get('Lat', 'no lat')
    lon = aircraft.get('Long', 'no lon')
    if(lat == 'no lat' or lon == 'no lon'):
        continue
    if(frm.lower().__contains__(target_from.lower()) and dest.lower().__contains__(target_dest.lower())):
        marker_location[0] = lat
        marker_location[1] = lon
        aircraft_info = [info_box_template.format(**aircraft)]
        marker = gmaps.marker_layer([marker_location], info_box_content=aircraft_info)
        map.add_layer(marker)

map


### Targets an aircraft by ID

In [ ]:
grab_data = True
if(grab_data):
    data = grabData()
target_ID = 10528990
alt = 0
map = gmaps.figure(map_type='HYBRID')
for aircraft in data['acList']:
    ID = aircraft.get('Id', 'no id')
    lat = aircraft.get('Lat', 'no lat')
    lon = aircraft.get('Long', 'no lon')
    alt = aircraft.get('Alt', 'no alt')
    spd = aircraft.get('Spd', 'no spd')
    if(lat == 'no lat' or lon == 'no lon'):
        continue
    if(ID == target_ID):
        marker_location[0] = lat
        marker_location[1] = lon
        aircraft_info = [info_box_template.format(**aircraft)]
        marker = gmaps.marker_layer([marker_location], info_box_content=aircraft_info)
        map.add_layer(marker)
     
dest_loc = [33.936527, -118.397243]
airportMarker = gmaps.marker_layer([dest_loc])
map.add_layer(airportMarker)

dist = distance(dest_loc[0], dest_loc[1], marker_location[0], marker_location[1])

print(round(dist, 4), 'miles to go')
print('Altitude:', alt)
print('Speed:', spd)

map


### Targets an aircraft model

In [ ]:
grab_data = True
if(grab_data):
    data = grabData()
target_model = 'Boeing 757-200'
map = gmaps.figure(map_type='HYBRID')
for aircraft in data['acList']:
    model = aircraft.get('Mdl', 'no mdl')
    lat = aircraft.get('Lat', 'no lat')
    lon = aircraft.get('Long', 'no lon')
    if(lat == 'no lat' or lon == 'no lon'):
        continue
    if(model.lower().__contains__(target_model.lower())):
        marker_location[0] = lat
        marker_location[1] = lon
        aircraft_info = [info_box_template.format(**aircraft)]
        marker = gmaps.marker_layer([marker_location], info_box_content=aircraft_info)
        map.add_layer(marker)

map
